In [1]:
import numpy as np
import mediapipe as mp
from PIL import Image
import string
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import cv2
from matplotlib import pyplot

# mp的solution定义
mp_drawing = mp.solutions.drawing_utils
mp_selfie_segmentation = mp.solutions.selfie_segmentation
# 选择预测用的模型
predModel = keras.models.load_model('models/model_res50.keras')
# 设定特征图大小
dim = (28, 28)
# 定义填充背景
BG_COLOR = (192, 192, 192)
# 定义默认识别框体位置
x0 = 100
x1 = 300
y0 = 110
y1 = 310

# 字典
dict_labels = {
    0: 'A',1: 'B',2: 'C',3: 'D',4: 'E',5: 'F',6: 'G',7: 'H',8: 'I',9: 'J',
    10: 'K',11: 'L',12: 'M',13: 'N',14: 'O',15: 'P',16: 'Q',17: 'R',18: 'S',
    19: 'T',20: 'U',21: 'V',22: 'W',23: 'X',24: 'Y',25: 'Z',    
}

# 初始化视频捕获
video=cv2.VideoCapture(0, cv2.CAP_DSHOW)
cv2.namedWindow('Masked By MediaPipe Segmentation') # 构建1个窗口
cv2.moveWindow('Masked By MediaPipe Segmentation',40,30) # 放置窗口
cv2.namedWindow('User Interface Camera') # 构建1个窗口
cv2.moveWindow('User Interface Camera',740,30) # 放置窗口

while video.isOpened(): # 只要没有关掉实时摄像头
    ret,capture = video.read() # 抓取每个视频帧
    capture = cv2.flip(capture,1)
    capture1=capture
    # 调用对应solotion
    with mp_selfie_segmentation.SelfieSegmentation(
        model_selection=1) as selfie_segmentation:
        # mediapipe色彩空间为RGB，故需要先进行转换
        image = cv2.cvtColor(capture, cv2.COLOR_BGR2RGB)
        # 将图像flags设定为不可写以提升性能
        image.flags.writeable = False
        # 处理
        results = selfie_segmentation.process(image)
        # 转回
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # 用np的堆叠法，使用结果中的mask定义双向过滤器
        condition = np.stack(
            (results.segmentation_mask,) * 3, axis=-1) > 0.1
        # 绘制背景图
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR
        # 过滤、合成图像
        capture = np.where(condition, image, bg_image)
    cropped = capture[y0:y1, x0:x1] # 截取
    img = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY) # 转成灰度图
    img = cv2.GaussianBlur(img, (3,3), 0) # 图像平滑
    img = cv2.resize(img, dim) # 图像大小缩放
    #pyplot.imshow(img, cmap='gray') # 可视化展示图片
    #pyplot.show() # 展示
    img = np.reshape(img, (1,img.shape[0],img.shape[1],1))
    img = tf.cast(img, tf.float32)
    pred = predModel.predict(img)

    # 可视化实时效果
    cv2.rectangle(capture, (x0,y0),(x1,y1),(255,0,0),2) # 为图片添加矩形框
    cv2.putText(capture,'{} '.format(dict_labels[pred.argmax(1).item()]),(x0+25,y0+50),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,255,0),1) # 预测字母
    cv2.imshow('Masked By MediaPipe Segmentation', capture) # 展示视频
    
    # 用户版
    cv2.rectangle(capture1, (x0,y0),(x1,y1),(255,0,0),2) # 为图片添加矩形框
    cv2.putText(capture1,'{} '.format(dict_labels[pred.argmax(1).item()]),(x0+25,y0+50),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,255,0),1) # 预测字母
    cv2.imshow('User Interface Camera', capture1) # 展示视频
    
    # mask版
    cv2.rectangle(results.segmentation_mask, (x0,y0),(x1,y1),(255,0,0),2) # 为图片添加矩形框
    cv2.putText(results.segmentation_mask,'{} '.format(dict_labels[pred.argmax(1).item()]),(x0+25,y0+50),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,255,0),1) # 预测字母
    cv2.imshow('mask', results.segmentation_mask) # 展示视频
    
    # 结果输出
    #print(pred)
    #print(dict_labels[pred.argmax(1).item()])
    
    # 退出视频输入
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

D:\Stu\CV\Z_Env\Anaconda3\envs\CV2023-QM\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Stu\CV\Z_Env\Anaconda3\envs\CV2023-QM\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
D:\Stu\CV\Z_Env\Anaconda3\envs\CV2023-QM\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
